# CS 195: Natural Language Processing
## Subword Tokenization and Byte Pair Encoding

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/s26-CS195NLP/blob/main/F2_4_SubwordTokenizationBPE.ipynb)


## Wrapping the output in Colab

Here's a resource on how to get Google Colab to wrap the output of a cell: https://stackoverflow.com/questions/58890109/line-wrapping-in-collaboratory-google-results

In short, put the following into a cell and run it:

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Announcement for next Monday

**First half of class:** NLP at Berkshire Hathaway Energy
* Vinay Kodigante
* Ryan Blokhuis (Drake alum)
* Amanda Pereira (Drake alum)

**Second half of class:** Normal demo day
* Does anyone have a *creative synthesis* project they're planning to show off that they'd like show our visitors?

**Before and during:** University Marketing and Communications may take some photos/videos for a social media post

<img src="images/JasonBHE.jpg"/>
New alumni hire: Jason Nguyen

## References

* [GPT Tokenizer Illustration](https://platform.openai.com/tokenizer)
* [Hugging Face Byte-Pair Encoding tokenization](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt)
* [Chapter 2: Words and Tokens](https://web.stanford.edu/~jurafsky/slp3/2.pdf). *Speech and Language Processing.* Daniel Jurafsky & James H. Martin
* Python resources for retrieving and tokenizing text data:
    * [Python `requests` library quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/)
    * [Beautiful Soup documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
    * [Python `split` method](https://docs.python.org/3/library/stdtypes.html#str.split)


In [26]:
#uncomment if you need to install things we're using today
#import sys
#!{sys.executable} -m pip install requests transformers beautifulsoup4

## What is a token?

A **token** is the unit of data that a language model operates on
* In our previous Markov Model, each word or punctuation mark counted as a token
* Tokens can also be parts of words (i.e., subwords) or even individual characters

**Tokenization** is the process of converting a stream of raw text into tokens - grouping it into groups of characters that represent a coherent unit

## Why do subword tokenization?

Natural langauge has a huge number of words (english has hundreds of thousands) and most are not frequently used

There common prefixes and suffixes, etc. that modify words in reliable ways

* think of a word like "hyperparameterization" which is "hyper" + "parameter" + "iz" + "ation"
* or "hyperfixation" = "hyper" + "fix" + "ation"
* or "organization" = "oran" + "iz" + "ation"

Mixtures of tokens gives the necessary meaning without having to create new tokens for every single variation

A model might be able to handle "hyperparameterization" without that word appearing in its training corpus


### Group Exercise

Try writing some sentences in the OpenAI tokenizer illustration: https://platform.openai.com/tokenizer

Discuss any interesting things you notice

## Unpacking the pipeline a little

Go to a Hugging Face model card and click the `Use this model` button near the upper-right part of the page

Example: https://huggingface.co/HuggingFaceTB/SmolLM2-135M

Notice that it separates the tokenizer from the model

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Using the tokenizer by itself

In [5]:
tokenized_text = tokenizer("Let's try to understand how SmolLM2 tokenizes its inputs.")
print(tokenized_text)

{'input_ids': [4239, 506, 1576, 288, 1044, 638, 3511, 308, 34519, 34, 9624, 3215, 624, 9577, 30], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


### Discussion

What do you notice?

In [7]:
tokenizer.convert_ids_to_tokens(tokenized_text['input_ids'])

['Let',
 "'s",
 'Ġtry',
 'Ġto',
 'Ġunderstand',
 'Ġhow',
 'ĠSm',
 'ol',
 'LM',
 '2',
 'Ġtoken',
 'izes',
 'Ġits',
 'Ġinputs',
 '.']

### Discussion

Now what do you notice?

## Let's investigate the same word written in different ways

In [9]:
more_experiments = tokenizer("Let's Let's let's\nLet's")
print(more_experiments)
print(tokenizer.convert_ids_to_tokens(more_experiments['input_ids']))

{'input_ids': [4239, 506, 2959, 506, 1303, 506, 198, 4239, 506], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
['Let', "'s", 'ĠLet', "'s", 'Ġlet', "'s", 'Ċ', 'Let', "'s"]


### Discussion

Now what do you notice?

## How do we use the tokenized data with the model object?

Underneath the `transformers` module is a lot of PyTorch code, which uses the `tensor` data type. 

A **tensor** is a multidimensional array, often representing a mathematical vector or matrix

Note that this is **not** the *instruct* version of the SmolLM2 model, so we can use it for text completion.

There's also a `decode` function that does all the work of converting the list of tokens back into readable text.

In [27]:
partial_sentence = "Let's try to understand how SmolLM tokenizes its"
partial_sentence_tokenized = tokenizer(partial_sentence, return_tensors="pt")
print("The 'tensor' version of the tokens:\n", partial_sentence_tokenized)
model_response = model.generate(
    input_ids = partial_sentence_tokenized["input_ids"], 
    attention_mask = partial_sentence_tokenized["attention_mask"]
)
print("The 'tensor' returned by the model:\n",model_response)
print("Using convert_ids_to_tokens:\n",tokenizer.convert_ids_to_tokens(model_response[0]))
print("Using decode:\n",tokenizer.decode(model_response[0]))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


The 'tensor' version of the tokens:
 {'input_ids': tensor([[ 4239,   506,  1576,   288,  1044,   638,  3511,   308, 34519,  9624,
          3215,   624]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
The 'tensor' returned by the model:
 tensor([[ 4239,   506,  1576,   288,  1044,   638,  3511,   308, 34519,  9624,
          3215,   624,   940,    30,   198,   198,  9207,   308, 34519,  9624,
          3215,   624,   940,   411,  1015,   253, 15345,  1517,    30,   378,
         15345,  1517]])
Using convert_ids_to_tokens:
 ['Let', "'s", 'Ġtry', 'Ġto', 'Ġunderstand', 'Ġhow', 'ĠSm', 'ol', 'LM', 'Ġtoken', 'izes', 'Ġits', 'Ġdata', '.', 'Ċ', 'Ċ', 'Sm', 'ol', 'LM', 'Ġtoken', 'izes', 'Ġits', 'Ġdata', 'Ġby', 'Ġusing', 'Ġa', 'Ġhash', 'Ġfunction', '.', 'ĠThe', 'Ġhash', 'Ġfunction']
Using decode:
 Let's try to understand how SmolLM tokenizes its data.

SmolLM tokenizes its data by using a hash function. The hash function


## Normalization and Pre-Tokenization

Text data often goes through a **normalization** step before tokenization
* removing extra whitespace
* converting to lowercase
* replace accented characters with unaccented (e.g., replace á with a)

**Pre-tokenization**
* separate characters into groups based on whitespace (and maybe punctuation)
* like when we used `split()` for our MarkovModel data


## Byte-Pair Encoding (BPE)

BPE is a tokenization algorithm
* Probably the most commonly used one (GPT, SmolLM, and many others)

BPE Training Algorithm Idea:
1. Pretokenize all of the things separated by whitespace, etc.
2. Start with a *vocabulary* containing all the individual characters from the pre-tokens
3. Until the vocabulary is the desired size
    * merge together the most frequently-appearing pair of tokens and add it as a new token to the *vocabulary*
  
Frequent words - don't break them apart

Less-frequent words - represent them as several subwords
    


## BPE Training Example

Let's go through the example from here: https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt

Assume the corpus has only the following words: "hug", "pug", "pun", "bun", "hugs" and they appear with these frequencies (meaning "hug" appears 10 times, "pug" appears 5 times, etc:

("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)

The initial vocabulary will be

In [16]:
vocabulary = ["b", "g", "h", "n", "p", "s", "u"]

Tokenize our pre-tokens using this vocabulary

In [17]:
corpus = [("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)]

Find how frequently each pair appears.

* "h" "u" appears 15 times (10 from "hug", 5 from "hugs")
* "u" "g" appears 20 times (10 from "hug", 5 from "pug", 5 from "hugs")
* "p" "u" appears __ times (exercise)
* "u" "n" appears __ times (exercise)
* "b" "u" appears __ times (exercise)
* "g" "s" appears __ times (exercise)

If "u" "g" is the most frequent, we then add it to our vocabulary. 

Vocabulary and corpus are now

In [18]:
vocabulary = ["b", "g", "h", "n", "p", "s", "u", "ug"]
corpus = [("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)]

Finding pair frequencies

* "h" "ug" appears 15 times (10 from "hug", 5 from "hugs")
* "p" "ug" appears 5 times
* "p" "u"  appears 12 times
* "u" "n"  appears 16 times
* "b" "u"  appears 4 times
* "ug" "s" appears 5 times

So we merge "u" "n"

In [19]:
vocabulary = ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]
corpus = [("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)]

Finding pair frequencies

* "h" "ug" appears 15 times
* "p" "ug" appears 5 times
* "p" "un"  appears 12 times
* "b" "un"  appears 4 times
* "ug" "s" appears 5 times

So we merge "h" "ug"

In [20]:
vocabulary = ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]
corpus = [("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)]

**Exercise:** when happens on the next merge step?

Remember - we stop when the vocabular hits a pre-determined size

## Tokenizing with BPE

The algorithm for tokenizing with this trained model (vocabulary) is

1. pre-tokenize (split by spaces)
2. split pre-tokens (words) into characters, use "[UNK]" for unknowns
3. apply the merge rules that were learned *in order*

In our example, we learned the rules
* ("u", "g") -> "ug"
* ("u", "n") -> "un"
* ("h", "ug") -> "hug"

Example text: "pun bug hugs mug"

In [21]:
vocabulary = ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]
pre_tokens = ["pun","bug","hugs","mug"]
split_pre_tokens = [["p","u","n"],["b","u","g"],["h","u","g","s"],["m","u","g"]]
split_pre_tokens = [["p","u","n"],["b","u","g"],["h","u","g","s"],["[UNK]","u","g"]] #there is no m in the vocabulary
after_first_merge_rule = [["p","u","n"],["b","ug"],["h","ug","s"],["[UNK]","ug"]]
after_second_merge_rule = [["p","un"],["b","ug"],["h","ug","s"],["[UNK]","ug"]]
after_third_merge_rule = [["p","un"],["b","ug"],["hug","s"],["[UNK]","ug"]] 
final_tokens = ["p","un","b","ug","hug","s","[UNK]","ug"]

Usually any character from the encoding system (UTF-8, Unicode, etc.) can be included in the vocabulary, but you might occasionally run into something non-standard and need to use `[UNK]`. 

## Applied Exploration

Find some new text, tokenize it according to one or more of the methods discussed here

Use it as input for the Markov Chain in the previous set of notes

Describe what you did and record notes about your results



## Extended Implementation Idea

Write the code that will implement a tokenization algorithm like BPE or WordPiece automatically

Both the BPE and WordPiece have examples in the Hugging Face course
* https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt
* https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt

Except: they use Hugging Face tokenizers for the pre-tokenization. You should do it using `split()` like we did. Because of the way that it groups tokens, I don't think you should have to worry about separating out punctuation, just include it in your vocabulary.

## Working with text you find on the web

Here's an example of using the `requests` library to load data from a website

In [28]:
import requests
from transformers import AutoTokenizer

response = requests.get("https://www.gutenberg.org/files/1661/1661-0.txt")
sherlock_raw_text = response.text

sherlock_tokens = tokenizer.tokenize( sherlock_raw_text )
print(sherlock_tokens[1000:1100])

['Ġactivity', ',', 'Ġhowever', ',', 'Ġwhich', 'ĠI', 'Ġmerely', 'Ġshared', 'Ġwith', 'Ġall', 'Ġthe', 'Ġreaders', 'Ġof', 'č', 'Ċ', 'the', 'Ġdaily', 'Ġpress', ',', 'ĠI', 'Ġknew', 'Ġlittle', 'Ġof', 'Ġmy', 'Ġformer', 'Ġfriend', 'Ġand', 'Ġcompanion', '.', 'čĊč', 'Ċ', 'One', 'Ġnight', 'âĢĶ', 'it', 'Ġwas', 'Ġon', 'Ġthe', 'Ġtwentieth', 'Ġof', 'ĠMarch', ',', 'Ġ', '1', '8', '8', '8', 'âĢĶ', 'I', 'Ġwas', 'Ġreturning', 'Ġfrom', 'Ġa', 'č', 'Ċ', 'jour', 'ney', 'Ġto', 'Ġa', 'Ġpatient', 'Ġ(', 'for', 'ĠI', 'Ġhad', 'Ġnow', 'Ġreturned', 'Ġto', 'Ġcivil', 'Ġpractice', '),', 'Ġwhen', 'č', 'Ċ', 'my', 'Ġway', 'Ġled', 'Ġme', 'Ġthrough', 'ĠBaker', 'ĠStreet', '.', 'ĠAs', 'ĠI', 'Ġpassed', 'Ġthe', 'Ġwell', '-', 'rem', 'embered', 'č', 'Ċ', 'door', ',', 'Ġwhich', 'Ġmust', 'Ġalways', 'Ġbe', 'Ġassociated', 'Ġin', 'Ġmy']


If the text is in `html` instead of plaintext, you can try to parse it out with the beautiful soup package.

In [33]:
from bs4 import BeautifulSoup
import requests
from transformers import AutoTokenizer

# wikipedia requires you to set a User Agent if you want to access the page in code
headers = {
    'User-Agent': 'DrakeNLPClassScraper/1.0.0'
}

response = requests.get("https://en.wikipedia.org/wiki/Sherlock_Holmes", headers=headers)
sherlock_wiki_html = BeautifulSoup(response.text, 'html.parser')

sherlock_wiki_text = sherlock_wiki_html.get_text()

sherlock_wiki_tokens = tokenizer.tokenize( sherlock_wiki_text[5000:5500] )


print(sherlock_wiki_tokens)




['d', 'Ġbi', 'ographer', ',', 'ĠDr', '.', 'ĠJohn', 'ĠH', '.', 'ĠWatson', ',', 'Ġwho', 'Ġusually', 'Ġaccompanies', 'ĠHolmes', 'Ġduring', 'Ġhis', 'Ġinvestigations', 'Ġand', 'Ġoften', 'Ġshares', 'Ġquarters', 'Ġwith', 'Ġhim', 'Ġat', 'Ġthe', 'Ġaddress', 'Ġof', 'Ġ', '2', '2', '1', 'B', 'ĠBaker', 'ĠStreet', ',', 'ĠLondon', ',', 'Ġwhere', 'Ġmany', 'Ġof', 'Ġthe', 'Ġstories', 'Ġbegin', '.', 'Ċ', 'Though', 'Ġnot', 'Ġthe', 'Ġfirst', 'Ġfictional', 'Ġdetective', ',', 'ĠSher', 'lock', 'ĠHolmes', 'Ġis', 'Ġarguably', 'Ġthe', 'Ġbest', 'Ġknown', '.[', '1', ']', 'ĠBy', 'Ġthe', 'Ġ', '1', '9', '9', '0', 's', ',', 'Ġover', 'Ġ', '2', '5', ',', '0', '0', '0', 'Ġstage', 'Ġadaptations', ',', 'Ġfilms', ',', 'Ġtelevision', 'Ġproductions', ',', 'Ġand', 'Ġpublications', 'Ġhad', 'Ġfeatured', 'Ġthe', 'Ġdetective', ',[', '2', ']', 'Ġand', 'ĠGuin', 'ness', 'ĠWorld', 'ĠRecords', 'Ġlists', 'Ġhim', 'Ġas', 'Ġthe', 'Ġmost', 'Ġportrayed', 'Ġhuman', 'Ġliterary', 'Ġcharacter']
